In [1]:
import datetime


import pandas as pd


from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, MSTL
#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut, MinTrace, ERM, Normality
from hierarchicalforecast.utils import aggregate

c:\Users\Morri\.conda\envs\capstone\lib\site-packages\statsforecast\core.py:26: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
df = pd.read_feather('../data/2018_tf.feather')
df


,borough,complaint_type,created_H,closed_H,agency,created_date,closed_date,status,resolution_action_updated_date,descriptor,...,open_bo_co,responded,completed,insufficient,jurisdiction,review,attempt,investigated,not,no
0,BRONX,Animal Abuse,2018-04-13 18:00:00,2018-04-13 18:00:00,NYPD,2018-04-13 18:22:30,2018-04-13 18:36:30,Closed,2018-04-13 18:36:30,Tortured,...,0.0,False,False,True,False,False,False,False,True,False
1,BRONX,Blocked Driveway,2018-04-13 18:00:00,2018-04-13 23:00:00,NYPD,2018-04-13 18:55:19,2018-04-13 23:20:04,Closed,2018-04-13 23:20:04,No Access,...,1.0,True,False,False,False,False,False,False,False,True
2,BRONX,Consumer Complaint,2018-04-13 18:00:00,2018-04-16 11:00:00,DCA,2018-04-13 18:28:46,2018-04-16 11:34:38,Closed,2018-04-16 11:34:38,Labor Violation,...,3.0,False,False,False,False,False,False,False,True,False
3,BRONX,Consumer Complaint,2018-04-13 18:00:00,2018-04-16 14:00:00,DCA,2018-04-13 18:25:13,2018-04-16 14:34:12,Closed,2018-04-16 14:34:12,Receipt Incomplete/Not Given,...,3.0,False,False,False,True,False,False,False,True,False
4,BRONX,Consumer Complaint,2018-04-13 18:00:00,2018-04-29 01:00:00,DCA,2018-04-13 18:21:40,2018-04-29 01:06:33,Closed,2018-04-29 01:06:33,Damaged/Defective Goods,...,3.0,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1886364,STATEN ISLAND,Noise - Residential,2018-12-31 23:00:00,2018-12-31 23:00:00,NYPD,2018-12-31 23:08:31,2018-12-31 23:18:44,Closed,2018-12-31 23:18:44,Loud Music/Party,...,9.0,True,False,False,False,False,False,False,False,False
1886365,STATEN ISLAND,Noise - Residential,2018-12-31 23:00:00,2019-01-01 04:00:00,NYPD,2018-12-31 23:39:46,2019-01-01 04:09:14,Closed,2019-01-01 04:09:14,Banging/Pounding,...,9.0,True,False,False,False,False,False,False,False,False
1886366,STATEN ISLAND,Noise - Residential,2018-12-31 23:00:00,2019-01-01 07:00:00,NYPD,2018-12-31 23:14:00,2019-01-01 07:40:25,Closed,2019-01-01 07:40:25,Loud Music/Party,...,9.0,True,False,False,False,False,False,False,False,True
1886367,STATEN ISLAND,Sewer,2018-12-31 23:00:00,2019-01-02 09:00:00,DEP,2018-12-31 23:36:00,2019-01-02 09:30:00,Closed,2019-01-02 09:30:00,Street Flooding (SJ),...,61.0,False,False,False,False,False,False,False,False,False


In [ ]:
# heirarchies = [
#     ['borough'],
#     [ 'borough','incident_zip'],
#     [ 'borough','incident_zip', 'agency'],
#     [ 'borough','incident_zip', 'agency', 'complaint_type'],
# ]

hdf = df[df['borough']!= 'Unspecified']
bdf, H_df, tags = aggregate(
    hdf,
    spec=[['agency','borough']]
)
hrec = HierarchicalReconciliation(
    reconcilers =
        [
            MinTrace('ols', nonnegative=True,num_threads=-1),
            # ERM('closed'),
            # BottomUp()
        ]
)
Y_df = bdf[(bdf['ds']>datetime.date(year=2018, month=10, day=1)) & (bdf['ds']<datetime.date(year=2018, month=12, day=1))]

models = [MSTL(
    season_length=[24, 24 * 7], # seasonalities of the time series
    trend_forecaster=AutoARIMA(trace=True) # model used to forecast trend
)]
sf = StatsForecast(
    models = models,
    freq = 'H',
    df=Y_df,
    n_jobs=-1
    )
sf.fit()
fcts  = sf.forecast(h=1,df=bdf, level=[90])
StatsForecast.plot(df=Y_df, forecasts_df=fcts[fcts.index.isin(bdf.index)].fillna(0))